# Exploratory Data Analysis with Python

BUSAN 302

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import plotly.offline as py
import plotly.express as px

import plotly.graph_objs as go
import plotly.figure_factory as ff
import cufflinks as cf

cf.set_config_file(offline=True, world_readable=False)

In [ ]:
# Unfortunately, Google colab needs us to call this function every time we make an offline plot
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

# Introduction

In this lecture we examine the process of data cleaning and Exploratory Data Analysis (EDA).  Often you will acquire or even be given a collection of data in order to conduct some analysis or answer some questions. The first step in using that data is to ensure that it is in the correct form (cleaned) and that you understand its properties and limitations (EDA).  Often as you explore data through EDA you will identify additional transformations that may be required before the data is ready for analysis.

In this notebook we obtain crime data from public records kept by the city of Berkeley in California.  Ultimately, our goal might be to understand policing patterns but before we get there we must first clean and understand the data. 

In [ ]:
!wget https://busan302.mycourses.work/code/busan302_utils.txt
!mv busan302_utils.txt busan302_utils.py
from busan302_utils import fetch_and_cache


--2022-08-09 04:08:48--  https://busan302.mycourses.work/code/busan302_utils.txt
Resolving busan302.mycourses.work (busan302.mycourses.work)... 173.236.164.245
Connecting to busan302.mycourses.work (busan302.mycourses.work)|173.236.164.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1645 (1.6K) [text/plain]
Saving to: ‘busan302_utils.txt’

busan302_utils.txt  100%[===================>]   1.61K  --.-KB/s    in 0s      

2022-08-09 04:08:49 (184 MB/s) - ‘busan302_utils.txt’ saved [1645/1645]



## Downloading the Data

Notice that because I record how I got the data in the notebook, others can reproduce this experiment.  However, it is worth noting that **the data can change**.  We will want to pay attention to file timestamps.

In [ ]:
calls_file = fetch_and_cache("https://busan302.mycourses.work/data/calls_for_service.csv", file="calls_for_service.csv", force=False)

Downloading... Done!


In [ ]:
import os
print(calls_file, "is",  os.path.getsize(calls_file) / 1e6, "MB")

data/calls_for_service.csv is 0.941358 MB


In [ ]:
with open(calls_file, "r") as f:
    print(calls_file, "is", sum(1 for l in f), "lines.")

data/calls_for_service.csv is 15452 lines.


In [ ]:
print(calls_file, "======================")
with open(calls_file, "r") as f:
    for i in range(10):
         #print(i, "\t", repr(f.readline()))
          print(i, "\t", (f.readline()))

data/calls_for_service.csv ======================
0 	 CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State

1 	 19092769,THEFT MISD. (UNDER $950),12/09/2019 12:00:00 AM,13:00,LARCENY,1,09/10/2020 07:00:11 AM,"SHATTUCK AVE

2 	 Berkeley, CA",SHATTUCK AVE,Berkeley,CA

3 	 19045891,NARCOTICS,08/18/2019 12:00:00 AM,17:20,DRUG VIOLATION,0,09/10/2020 07:00:08 AM,"FRONTAGE STREET &GILMAN ST

4 	 Berkeley, CA",FRONTAGE STREET &GILMAN ST,Berkeley,CA

5 	 19060215,ASSAULT/BATTERY MISD.,10/23/2019 12:00:00 AM,10:45,ASSAULT,3,09/10/2020 07:00:10 AM,"2200 MILVIA ST

6 	 Berkeley, CA

7 	 (37.868574, -122.270415)",2200 MILVIA ST,Berkeley,CA

8 	 19092681,VANDALISM,12/01/2019 12:00:00 AM,18:40,VANDALISM,0,09/10/2020 07:00:11 AM,"VIRGINIA ST

9 	 Berkeley, CA",VIRGINIA ST,Berkeley,CA




### Loading the Calls Data

Because the file appears to be a relatively well formatted CSV we will attempt to load it directly and allow the Pandas Library to deduce column headers.  (Always check that first row and column look correct after loading.)

In [ ]:
calls = pd.read_csv(calls_file, warn_bad_lines=True)
calls.head(10)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning:

The warn_bad_lines argument has been deprecated and will be removed in a future version.





,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State
0,19092769,THEFT MISD. (UNDER $950),12/09/2019 12:00:00 AM,13:00,LARCENY,1,09/10/2020 07:00:11 AM,"SHATTUCK AVE\nBerkeley, CA",SHATTUCK AVE,Berkeley,CA
1,19045891,NARCOTICS,08/18/2019 12:00:00 AM,17:20,DRUG VIOLATION,0,09/10/2020 07:00:08 AM,"FRONTAGE STREET &GILMAN ST\nBerkeley, CA",FRONTAGE STREET &GILMAN ST,Berkeley,CA
2,19060215,ASSAULT/BATTERY MISD.,10/23/2019 12:00:00 AM,10:45,ASSAULT,3,09/10/2020 07:00:10 AM,"2200 MILVIA ST\nBerkeley, CA\n(37.868574, -122...",2200 MILVIA ST,Berkeley,CA
3,19092681,VANDALISM,12/01/2019 12:00:00 AM,18:40,VANDALISM,0,09/10/2020 07:00:11 AM,"VIRGINIA ST\nBerkeley, CA",VIRGINIA ST,Berkeley,CA
4,19044228,ASSAULT/BATTERY MISD.,08/10/2019 12:00:00 AM,22:51,ASSAULT,6,09/10/2020 07:00:08 AM,"UNIVERSITY AVENUE &FRONTAGE\nBerkeley, CA",UNIVERSITY AVENUE &FRONTAGE,Berkeley,CA
5,19092551,THEFT MISD. (UNDER $950),11/17/2019 12:00:00 AM,12:00,LARCENY,0,09/10/2020 07:00:11 AM,"ASHBY AVE\nBerkeley, CA",ASHBY AVE,Berkeley,CA
6,19047517,BURGLARY AUTO,08/25/2019 12:00:00 AM,18:25,BURGLARY - VEHICLE,0,09/10/2020 07:00:08 AM,"CATALINA AVE\nBerkeley, CA",CATALINA AVE,Berkeley,CA
7,19091711,VANDALISM,08/19/2019 12:00:00 AM,22:00,VANDALISM,1,09/10/2020 07:00:08 AM,"CALIFORNIA STREET & FAIRVIEW ST\nBerkeley, CA",CALIFORNIA STREET & FAIRVIEW ST,Berkeley,CA
8,19092111,VANDALISM,09/24/2019 12:00:00 AM,20:00,VANDALISM,2,09/10/2020 07:00:09 AM,"600 CANYON RD\nBerkeley, CA",600 CANYON RD,Berkeley,CA
9,19091461,THEFT MISD. (UNDER $950),07/23/2019 12:00:00 AM,14:05,LARCENY,2,09/10/2020 07:00:08 AM,"1700 FOURTH ST\nBerkeley, CA\n(37.871883, -122...",1700 FOURTH ST,Berkeley,CA


How many records did we get?

In [ ]:
calls.shape[0]

5227

### Preliminary observations on the data?

1. `EVENTDT` -- Contain the incorrect time stamp
1. `EVENTTM` -- Contains the time in 24 hour format (What timezone?)
1. `CVDOW` -- Appears to be some encoding of the day of the week (see data documentation).
1. `InDbDate` -- Appears to be correctly formatted and appears pretty consistent in time.
1. **`Block_Location` -- Errr, what a mess!  newline characters, and Geocoordinates all merged!!  Fortunately, this field was "quoted" otherwise we would have had trouble parsing the file. (why?)**
1. `BLKADDR` -- This appears to be the address in Block Location.
1. `City` and `State` seem redundant given this is supposed to be the city of Berkeley dataset.


### Checking that the City and State fields are all Berkeley CA

We notice that there are city and state columns.  Since this is supposed to be data for the city of Berkeley these columns appear to be redundant.  Let's quickly compute the number of occurences of unique values for these two columns.

In [ ]:
calls.groupby(["City", "State"]).count()

,,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR
City,State,,,,,,,,,
Berkeley,CA,5227,5227,5227,5227,5227,5227,5227,5227,5190


### Decoding day of the week

According to the documentation `CVDOW=0` is Sunday, `CVDOW=1` is Monday, ...,  Therefore we can make a series to decode the day of the week for each record and join that series with the calls data.

In [ ]:
dow = pd.Series(["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"], name="Day")
dow

0       Sunday
1       Monday
2      Tuesday
3    Wednesday
4     Thursday
5       Friday
6     Saturday
Name: Day, dtype: object

In [ ]:
df_dow = pd.DataFrame(dow)
# Notice that I am dropping the column if it already exists to
# make it so I can run this cell more than once
calls = pd.merge(calls.drop(columns="Day", errors="ignore"), 
         df_dow, left_on='CVDOW', right_index=True).sort_index()
calls

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Day
0,19092769,THEFT MISD. (UNDER $950),12/09/2019 12:00:00 AM,13:00,LARCENY,1,09/10/2020 07:00:11 AM,"SHATTUCK AVE\nBerkeley, CA",SHATTUCK AVE,Berkeley,CA,Monday
1,19045891,NARCOTICS,08/18/2019 12:00:00 AM,17:20,DRUG VIOLATION,0,09/10/2020 07:00:08 AM,"FRONTAGE STREET &GILMAN ST\nBerkeley, CA",FRONTAGE STREET &GILMAN ST,Berkeley,CA,Sunday
2,19060215,ASSAULT/BATTERY MISD.,10/23/2019 12:00:00 AM,10:45,ASSAULT,3,09/10/2020 07:00:10 AM,"2200 MILVIA ST\nBerkeley, CA\n(37.868574, -122...",2200 MILVIA ST,Berkeley,CA,Wednesday
3,19092681,VANDALISM,12/01/2019 12:00:00 AM,18:40,VANDALISM,0,09/10/2020 07:00:11 AM,"VIRGINIA ST\nBerkeley, CA",VIRGINIA ST,Berkeley,CA,Sunday
4,19044228,ASSAULT/BATTERY MISD.,08/10/2019 12:00:00 AM,22:51,ASSAULT,6,09/10/2020 07:00:08 AM,"UNIVERSITY AVENUE &FRONTAGE\nBerkeley, CA",UNIVERSITY AVENUE &FRONTAGE,Berkeley,CA,Saturday
...,...,...,...,...,...,...,...,...,...,...,...,...
5222,19092686,BURGLARY AUTO,12/02/2019 12:00:00 AM,08:30,BURGLARY - VEHICLE,1,09/10/2020 07:00:11 AM,"1000 GILMAN ST\nBerkeley, CA\n(37.88014, -122....",1000 GILMAN ST,Berkeley,CA,Monday
5223,19043965,THEFT MISD. (UNDER $950),08/05/2019 12:00:00 AM,23:30,LARCENY,1,09/10/2020 07:00:08 AM,"2300 WARD ST\nBerkeley, CA\n(37.860105, -122.2...",2300 WARD ST,Berkeley,CA,Monday
5224,19052923,BURGLARY AUTO,09/19/2019 12:00:00 AM,13:30,BURGLARY - VEHICLE,4,09/10/2020 07:00:09 AM,"1500 SHATTUCK AV\nBerkeley, CA\n(37.880227, -1...",1500 SHATTUCK AV,Berkeley,CA,Thursday
5225,19049410,DISTURBANCE,08/05/2019 12:00:00 AM,10:00,DISORDERLY CONDUCT,1,09/10/2020 07:00:09 AM,"2900 REGENT ST\nBerkeley, CA\n(37.857787, -122...",2900 REGENT ST,Berkeley,CA,Monday


### Cleaning Block Location

The block location contains the lat/lon coordinates and I might want to use these to analyze the location of each request.  Let's try to extract the GPS coordinates using regular expressions (we will cover regular expressions in future lectures):


In [ ]:
calls['Block_Location'].head(10)

0                           SHATTUCK AVE\nBerkeley, CA
1             FRONTAGE STREET &GILMAN ST\nBerkeley, CA
2    2200 MILVIA ST\nBerkeley, CA\n(37.868574, -122...
3                            VIRGINIA ST\nBerkeley, CA
4            UNIVERSITY AVENUE &FRONTAGE\nBerkeley, CA
5                              ASHBY AVE\nBerkeley, CA
6                           CATALINA AVE\nBerkeley, CA
7        CALIFORNIA STREET & FAIRVIEW ST\nBerkeley, CA
8                          600 CANYON RD\nBerkeley, CA
9    1700 FOURTH ST\nBerkeley, CA\n(37.871883, -122...
Name: Block_Location, dtype: object

In [ ]:
"""
Syntax: Series.str.extract(pat, flags=0, expand=True)

Parameter :
pat : Regular expression pattern with capturing groups.
flags : int, default 0 (no flags)
expand : If True, return DataFrame with one column per capture group.

Returns : DataFrame or Series or Index

A Regular Expression (or Regex) is a pattern (or filter) that describes a set of strings that matches the pattern. In other words, a regex accepts a certain set of strings and rejects the rest.

find some common regular expression here: https://www.w3schools.com/python/python_regex.asp
"""
calls_lat_lon = (
    # Remove newlines
    calls['Block_Location'].str.replace("\n", "\t") 
    # Extract Lat and Lon using regular expression
    .str.extract(".*\((?P<Lat>\d*\.\d*)\, (?P<Lon>-?\d*\.\d*)\)", expand=True)
)
calls_lat_lon.head(20)

,Lat,Lon
0,NaN,NaN
1,NaN,NaN
2,37.868574,-122.270415
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,37.871883,-122.301255


Not all the records have a lat and lon.  What fraction do have coordinates?

In [ ]:
(~calls_lat_lon.isnull()).mean()

Lat    0.963076
Lon    0.963076
dtype: float64

The following block of code joins the extracted Latitude and Longitude fields with the calls data.  Notice that we actually drop these fields before joining.  This is to enable repeated invocation of this cell even after the join has been completed. 

In [ ]:
# Remove Lat and Lon if they already existed before (reproducible)
calls.drop(["Lat", "Lon"], axis=1, inplace=True, errors="ignore")
# Join in the the latitude and longitude data
calls = calls.merge(calls_lat_lon, left_index=True, right_index=True)
# calls[["Lat", "Lon"]] = calls_lat_lon
# calls.join(calls_lat_lon)
calls.head(10)

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Day,Lat,Lon
0,19092769,THEFT MISD. (UNDER $950),12/09/2019 12:00:00 AM,13:00,LARCENY,1,09/10/2020 07:00:11 AM,"SHATTUCK AVE\nBerkeley, CA",SHATTUCK AVE,Berkeley,CA,Monday,NaN,NaN
1,19045891,NARCOTICS,08/18/2019 12:00:00 AM,17:20,DRUG VIOLATION,0,09/10/2020 07:00:08 AM,"FRONTAGE STREET &GILMAN ST\nBerkeley, CA",FRONTAGE STREET &GILMAN ST,Berkeley,CA,Sunday,NaN,NaN
2,19060215,ASSAULT/BATTERY MISD.,10/23/2019 12:00:00 AM,10:45,ASSAULT,3,09/10/2020 07:00:10 AM,"2200 MILVIA ST\nBerkeley, CA\n(37.868574, -122...",2200 MILVIA ST,Berkeley,CA,Wednesday,37.868574,-122.270415
3,19092681,VANDALISM,12/01/2019 12:00:00 AM,18:40,VANDALISM,0,09/10/2020 07:00:11 AM,"VIRGINIA ST\nBerkeley, CA",VIRGINIA ST,Berkeley,CA,Sunday,NaN,NaN
4,19044228,ASSAULT/BATTERY MISD.,08/10/2019 12:00:00 AM,22:51,ASSAULT,6,09/10/2020 07:00:08 AM,"UNIVERSITY AVENUE &FRONTAGE\nBerkeley, CA",UNIVERSITY AVENUE &FRONTAGE,Berkeley,CA,Saturday,NaN,NaN
5,19092551,THEFT MISD. (UNDER $950),11/17/2019 12:00:00 AM,12:00,LARCENY,0,09/10/2020 07:00:11 AM,"ASHBY AVE\nBerkeley, CA",ASHBY AVE,Berkeley,CA,Sunday,NaN,NaN
6,19047517,BURGLARY AUTO,08/25/2019 12:00:00 AM,18:25,BURGLARY - VEHICLE,0,09/10/2020 07:00:08 AM,"CATALINA AVE\nBerkeley, CA",CATALINA AVE,Berkeley,CA,Sunday,NaN,NaN
7,19091711,VANDALISM,08/19/2019 12:00:00 AM,22:00,VANDALISM,1,09/10/2020 07:00:08 AM,"CALIFORNIA STREET & FAIRVIEW ST\nBerkeley, CA",CALIFORNIA STREET & FAIRVIEW ST,Berkeley,CA,Monday,NaN,NaN
8,19092111,VANDALISM,09/24/2019 12:00:00 AM,20:00,VANDALISM,2,09/10/2020 07:00:09 AM,"600 CANYON RD\nBerkeley, CA",600 CANYON RD,Berkeley,CA,Tuesday,NaN,NaN
9,19091461,THEFT MISD. (UNDER $950),07/23/2019 12:00:00 AM,14:05,LARCENY,2,09/10/2020 07:00:08 AM,"1700 FOURTH ST\nBerkeley, CA\n(37.871883, -122...",1700 FOURTH ST,Berkeley,CA,Tuesday,37.871883,-122.301255


We can now look at a few of the records that were missing latitude and longitude entries:

In [ ]:
calls[calls['Lat'].isnull()].head(10)

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Day,Lat,Lon
0,19092769,THEFT MISD. (UNDER $950),12/09/2019 12:00:00 AM,13:00,LARCENY,1,09/10/2020 07:00:11 AM,"SHATTUCK AVE\nBerkeley, CA",SHATTUCK AVE,Berkeley,CA,Monday,NaN,NaN
1,19045891,NARCOTICS,08/18/2019 12:00:00 AM,17:20,DRUG VIOLATION,0,09/10/2020 07:00:08 AM,"FRONTAGE STREET &GILMAN ST\nBerkeley, CA",FRONTAGE STREET &GILMAN ST,Berkeley,CA,Sunday,NaN,NaN
3,19092681,VANDALISM,12/01/2019 12:00:00 AM,18:40,VANDALISM,0,09/10/2020 07:00:11 AM,"VIRGINIA ST\nBerkeley, CA",VIRGINIA ST,Berkeley,CA,Sunday,NaN,NaN
4,19044228,ASSAULT/BATTERY MISD.,08/10/2019 12:00:00 AM,22:51,ASSAULT,6,09/10/2020 07:00:08 AM,"UNIVERSITY AVENUE &FRONTAGE\nBerkeley, CA",UNIVERSITY AVENUE &FRONTAGE,Berkeley,CA,Saturday,NaN,NaN
5,19092551,THEFT MISD. (UNDER $950),11/17/2019 12:00:00 AM,12:00,LARCENY,0,09/10/2020 07:00:11 AM,"ASHBY AVE\nBerkeley, CA",ASHBY AVE,Berkeley,CA,Sunday,NaN,NaN
6,19047517,BURGLARY AUTO,08/25/2019 12:00:00 AM,18:25,BURGLARY - VEHICLE,0,09/10/2020 07:00:08 AM,"CATALINA AVE\nBerkeley, CA",CATALINA AVE,Berkeley,CA,Sunday,NaN,NaN
7,19091711,VANDALISM,08/19/2019 12:00:00 AM,22:00,VANDALISM,1,09/10/2020 07:00:08 AM,"CALIFORNIA STREET & FAIRVIEW ST\nBerkeley, CA",CALIFORNIA STREET & FAIRVIEW ST,Berkeley,CA,Monday,NaN,NaN
8,19092111,VANDALISM,09/24/2019 12:00:00 AM,20:00,VANDALISM,2,09/10/2020 07:00:09 AM,"600 CANYON RD\nBerkeley, CA",600 CANYON RD,Berkeley,CA,Tuesday,NaN,NaN
67,19043111,DISTURBANCE,08/06/2019 12:00:00 AM,08:55,DISORDERLY CONDUCT,2,09/10/2020 07:00:08 AM,"2134 MARTIN LUTHER KING JUNIOR WAY\nBerkeley, CA",2134 MARTIN LUTHER KING JUNIOR WAY,Berkeley,CA,Tuesday,NaN,NaN
164,19066087,DISTURBANCE,11/20/2019 12:00:00 AM,14:14,DISORDERLY CONDUCT,3,09/10/2020 07:00:11 AM,MARTIN LUTHER KING JR WAY &ALCATRAZ AVE\nBerke...,MARTIN LUTHER KING JR WAY &ALCATRAZ AVE,Berkeley,CA,Wednesday,NaN,NaN


Are there any patterns to the missing data?

---
<br/><br/><br/>

### Are Case Numbers unique?

Case numbers are probably used internally to track individual cases and my reference other data we don't have access to.  However, it is possible that multiple calls could be associated with the same case.  Let's see if the case numbers are all unique.

In [ ]:
print("There are", calls['CASENO'].unique().shape[0], "unique case numbers.")
print("There are", calls.shape[0], "calls in the table.")

There are 5227 unique case numbers.
There are 5227 calls in the table.


Are case numbers assigned consecutively.  

In [ ]:
configure_plotly_browser_state()

calls['CASENO'].sort_values().reset_index(drop=True).iplot(
    yTitle="Case Number", xTitle="Location in File")

I like to use interactive plotting tools so I can hover the mouse over the plot and read the values.  The cufflinks library adds plotly support to Pandas.  

### What might we be observing?

It looks like there are three discrete regions - an initial value, a consecutive increasing range, a slight gap(!) and another consecutive increasing range.

In [ ]:
calls['CASENO'].sort_values().head()

4121    17043531
2204    19010435
4893    19013597
3402    19035723
1587    19035759
Name: CASENO, dtype: int64

Let's look at row 4121.

In [ ]:
calls.iloc[[4121]]

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Day,Lat,Lon
4121,17043531,SEXUAL ASSAULT FEL.,07/18/2019 12:00:00 AM,00:00,SEX CRIME,4,09/10/2020 07:00:08 AM,"UNKNOWN\nBerkeley, CA",UNKNOWN,Berkeley,CA,Thursday,NaN,NaN


This record looks very anomalous as it is missing values for multiple important fields.

---
<br/><br/><br/>

## Examining the Date

Let's dig into the date in which events were recorded.  Notice in this data we have several pieces of date/time information (this is not uncommon):
1. **`EVENTDT`**: This contains the date the event took place.  While it has time information the time appears to be `00:00:00`.  
1. **`EVENTTM`**: This contains the time at which the event took place.
1. **`InDbDate`**: This appears to be the date at which the data was entered in the database.  

In [ ]:
calls.head(3)

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Day,Lat,Lon
0,19092769,THEFT MISD. (UNDER $950),12/09/2019 12:00:00 AM,13:00,LARCENY,1,09/10/2020 07:00:11 AM,"SHATTUCK AVE\nBerkeley, CA",SHATTUCK AVE,Berkeley,CA,Monday,NaN,NaN
1,19045891,NARCOTICS,08/18/2019 12:00:00 AM,17:20,DRUG VIOLATION,0,09/10/2020 07:00:08 AM,"FRONTAGE STREET &GILMAN ST\nBerkeley, CA",FRONTAGE STREET &GILMAN ST,Berkeley,CA,Sunday,NaN,NaN
2,19060215,ASSAULT/BATTERY MISD.,10/23/2019 12:00:00 AM,10:45,ASSAULT,3,09/10/2020 07:00:10 AM,"2200 MILVIA ST\nBerkeley, CA\n(37.868574, -122...",2200 MILVIA ST,Berkeley,CA,Wednesday,37.868574,-122.270415


When Pandas loads more complex fields like dates it will often load them as strings:

In [ ]:
calls["EVENTDT"][0]

'12/09/2019 12:00:00 AM'

We will want to convert these to dates.  Pandas has a fairly sophisticated function `pd.to_datetime` which is capable of guessing reasonable conversions of dates to date objects. 

In [ ]:
dates = pd.to_datetime(calls["EVENTDT"])
dates[0]

Timestamp('2019-12-09 00:00:00')

We can verify that the translations worked by looking at a few dates:

In [ ]:
pd.DataFrame(dict(transformed=dates, original=calls["EVENTDT"])).head()

,transformed,original
0,2019-12-09,12/09/2019 12:00:00 AM
1,2019-08-18,08/18/2019 12:00:00 AM
2,2019-10-23,10/23/2019 12:00:00 AM
3,2019-12-01,12/01/2019 12:00:00 AM
4,2019-08-10,08/10/2019 12:00:00 AM


We can also extract the time field:

In [ ]:
times = pd.to_datetime(calls["EVENTTM"]).dt.time
times.head()

0    13:00:00
1    17:20:00
2    10:45:00
3    18:40:00
4    22:51:00
Name: EVENTTM, dtype: object

To combine the correct date and correct time field we use the built-in python datetime combine function.

In [ ]:
from datetime import datetime
timestamps = pd.concat([dates, times], axis=1).apply(
    lambda r: datetime.combine(r['EVENTDT'], r['EVENTTM']), axis=1)
timestamps.head()

0   2019-12-09 13:00:00
1   2019-08-18 17:20:00
2   2019-10-23 10:45:00
3   2019-12-01 18:40:00
4   2019-08-10 22:51:00
dtype: datetime64[ns]

We now updated calls to contain this additional informations:

In [ ]:
calls['timestamp'] = timestamps
calls.head()

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State,Day,Lat,Lon,timestamp
0,19092769,THEFT MISD. (UNDER $950),12/09/2019 12:00:00 AM,13:00,LARCENY,1,09/10/2020 07:00:11 AM,"SHATTUCK AVE\nBerkeley, CA",SHATTUCK AVE,Berkeley,CA,Monday,NaN,NaN,2019-12-09 13:00:00
1,19045891,NARCOTICS,08/18/2019 12:00:00 AM,17:20,DRUG VIOLATION,0,09/10/2020 07:00:08 AM,"FRONTAGE STREET &GILMAN ST\nBerkeley, CA",FRONTAGE STREET &GILMAN ST,Berkeley,CA,Sunday,NaN,NaN,2019-08-18 17:20:00
2,19060215,ASSAULT/BATTERY MISD.,10/23/2019 12:00:00 AM,10:45,ASSAULT,3,09/10/2020 07:00:10 AM,"2200 MILVIA ST\nBerkeley, CA\n(37.868574, -122...",2200 MILVIA ST,Berkeley,CA,Wednesday,37.868574,-122.270415,2019-10-23 10:45:00
3,19092681,VANDALISM,12/01/2019 12:00:00 AM,18:40,VANDALISM,0,09/10/2020 07:00:11 AM,"VIRGINIA ST\nBerkeley, CA",VIRGINIA ST,Berkeley,CA,Sunday,NaN,NaN,2019-12-01 18:40:00
4,19044228,ASSAULT/BATTERY MISD.,08/10/2019 12:00:00 AM,22:51,ASSAULT,6,09/10/2020 07:00:08 AM,"UNIVERSITY AVENUE &FRONTAGE\nBerkeley, CA",UNIVERSITY AVENUE &FRONTAGE,Berkeley,CA,Saturday,NaN,NaN,2019-08-10 22:51:00


### What time range does the data represent

In [ ]:
calls['timestamp'].min()

Timestamp('2019-07-01 00:00:00')

In [ ]:
calls['timestamp'].max()

Timestamp('2019-12-20 01:30:00')

---
<br/><br/><br/>

### CVLEGEND

The CVLEGEND field provides the broad category of crime and is a good mechanism to group potentially similar crimes. 

In [ ]:
configure_plotly_browser_state()

calls['CVLEGEND'].value_counts().iplot(kind="bar")

Notice that when we group by the crime time we see that **larceny** emerges as one of the top crimes.  Larceny is essentially stealing -- taking someone else stuff without force.

## Examining Location information

Let's examine the geographic data (latitude and longitude).  Recall that we had some missing values.  Let's look at the behavior of these missing values according to crime type.

In [ ]:
configure_plotly_browser_state()

calls["missing_lat_lon"] = calls[['Lat', 'Lon']].isnull().any(axis=1)
calls.groupby("CVLEGEND")['missing_lat_lon'].mean().sort_values().iplot(kind="barh")

---
<br/><br/><br/> 

### Examine data geographically


In [ ]:
import folium
import folium.plugins # The Folium Javascript Map Library

SF_COORDINATES = (37.87, -122.28)
sf_map = folium.Map(location=SF_COORDINATES, zoom_start=13)
locs = calls[['Lat', 'Lon']].astype('float').dropna().to_numpy()
heatmap = folium.plugins.HeatMap(locs.tolist(), radius = 10)
sf_map.add_child(heatmap)


### Questions

1. Why are all the calls located on the street and at often at intersections?


In [ ]:
cluster = folium.plugins.MarkerCluster()
for _, r in calls[['Lat', 'Lon', 'CVLEGEND']].tail(1000).dropna().iterrows():
    cluster.add_child(
        folium.Marker([float(r["Lat"]), float(r["Lon"])], popup=r['CVLEGEND']))
    
sf_map = folium.Map(location=SF_COORDINATES, zoom_start=13)
sf_map.add_child(cluster)
sf_map